In [1]:
!pip install transformers==3.0.2 essential_generators

# Test sentence embedding with local deployment of "Roberta-base" model

In [2]:
%%time

import json
import torch
from torch import nn
# Roberta Model and Tokenizer
from transformers import RobertaModel, RobertaTokenizer, AutoTokenizer, BertModel
import os
import pickle

CPU times: user 5.05 s, sys: 5.74 s, total: 10.8 s
Wall time: 918 ms


In [3]:
%%time

# Initialize tokenizer and model from HuggingFaces
JSON_CONTENT_TYPE = 'application/json'
PRE_TRAINED_MODEL_NAME = 'roberta-base'
# CLASS_NAMES = ['negative', 'neutral', 'positive']
tokenizer = RobertaTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)
model = RobertaModel.from_pretrained(PRE_TRAINED_MODEL_NAME)
MAX_LEN = 160

CPU times: user 4.31 s, sys: 739 ms, total: 5.04 s
Wall time: 4.54 s


In [4]:
%%time

test_data = {"text": "AWS is excited to announce that TorchServe is natively supported in Amazon SageMaker as the default model server for PyTorch inference"}
review_text = test_data['text']
print(review_text)

AWS is excited to announce that TorchServe is natively supported in Amazon SageMaker as the default model server for PyTorch inference
CPU times: user 40 µs, sys: 27 µs, total: 67 µs
Wall time: 65.8 µs


In [5]:
%%time

encoded_review = tokenizer.encode_plus(
                                        review_text,
                                        max_length=MAX_LEN,
                                        add_special_tokens=True,
                                        return_token_type_ids=False,
                                        padding=True,
                                        return_attention_mask=True,
                                        return_tensors='pt',
                                        )

input_ids = encoded_review['input_ids']
attention_mask = encoded_review['attention_mask']

CPU times: user 1.32 ms, sys: 0 ns, total: 1.32 ms
Wall time: 1.25 ms


In [6]:
%%time

output = model(input_ids, attention_mask)

embedding = pickle.dumps(output)
decoded_embedding = pickle.loads(embedding)

print(embedding)

b'\x80\x03ctorch._utils\n_rebuild_tensor_v2\nq\x00(ctorch.storage\n_load_from_bytes\nq\x01B\xfch\x01\x00\x80\x02\x8a\nl\xfc\x9cF\xf9 j\xa8P\x19.\x80\x02M\xe9\x03.\x80\x02}q\x00(X\x10\x00\x00\x00protocol_versionq\x01M\xe9\x03X\r\x00\x00\x00little_endianq\x02\x88X\n\x00\x00\x00type_sizesq\x03}q\x04(X\x05\x00\x00\x00shortq\x05K\x02X\x03\x00\x00\x00intq\x06K\x04X\x04\x00\x00\x00longq\x07K\x04uu.\x80\x02(X\x07\x00\x00\x00storageq\x00ctorch\nFloatStorage\nq\x01X\x0e\x00\x00\x0094900819114336q\x02X\x03\x00\x00\x00cpuq\x03M\x00ZNtq\x04Q.\x80\x02]q\x00X\x0e\x00\x00\x0094900819114336q\x01a.\x00Z\x00\x00\x00\x00\x00\x00gF\xa1\xbc\x82#\xc9=\xd8O\x13\xbd\xb0\xff\x9b\xbd\xafT\xa3=\x9a\xcd\xb3\xbd\xad\x90\xe3\xbc\xfch\x1f;\x92\xc9|=\xfe\x00;\xbd\x04\xfb\x07\xbd\xf8\xb1\x03=\x86\xa6\x90<\xbd\xec\xa8\xbc)+:=\xf9%;=\x80\x90\x84\xbd\xbeT\xf5\xbc=\x07q<{\xf4\xa3\xbb\xc7\x9b\xdf\xbdr\xaeC=\x99[#\xbd\xee\xfdm=7$\xdd<I\x11!=Q1\xac=b\xea\xbe=P4\x94\xbd\x12>\x0c;\x95\xcaX\xbb\x08\xbf=\xbd\xcf\xc1\xe6\xbb\xdd\x

## Save HuggingFaces model and tokenizer assets into local directory

In [7]:
%%bash
if [ -d "local_model_directory" ]; then
    rm -f local_model_directory/*    
else
    mkdir local_model_directory
fi

In [8]:
import os.path

local_model_path_var = os.path.join('./', 'local_model_directory')

tokenizer.save_pretrained(local_model_path_var)
model.save_pretrained(local_model_path_var)

In [9]:
!ls -alh {local_model_path_var}

total 477M
drwxrwxr-x 2 ec2-user ec2-user 4.0K Apr  7 15:59 .
drwxrwxr-x 5 ec2-user ec2-user 4.0K Apr  7 15:59 ..
-rw-rw-r-- 1 ec2-user ec2-user  510 Apr  7 15:59 config.json
-rw-rw-r-- 1 ec2-user ec2-user 446K Apr  7 15:59 merges.txt
-rw-rw-r-- 1 ec2-user ec2-user 476M Apr  7 15:59 pytorch_model.bin
-rw-rw-r-- 1 ec2-user ec2-user  772 Apr  7 15:59 special_tokens_map.json
-rw-rw-r-- 1 ec2-user ec2-user   25 Apr  7 15:59 tokenizer_config.json
-rw-rw-r-- 1 ec2-user ec2-user 878K Apr  7 15:59 vocab.json


In [10]:
#Replace S3 Bucket
%% sh

s3_bucket='eks-sagemaker-us-west-2-bucket'

(cd local_model_directory ; tar -czvf roberta-base.tar.gz *)
mv local_model_directory/roberta-base.tar.gz . 
aws s3 cp roberta-base.tar.gz s3://$s3_bucket/roberta/

# Create 3000 copies of roberta-base model

In [11]:
%%sh

#Replace S3 Bucket

s3_bucket='eks-sagemaker-us-west-2-bucket'

for i in {0..2999}
do
  aws s3 cp roberta-base.tar.gz s3://$s3_bucket/roberta/roberta-base-$i.tar.gz 
done

In [12]:
!aws s3 ls s3://eks-sagemaker-us-west-2-bucket/roberta/

2021-04-07 01:58:28  295324092 roberta-base-0.tar.gz
2021-04-07 01:58:30  295324092 roberta-base-1.tar.gz
2021-04-07 01:59:09  295324092 roberta-base-10.tar.gz
2021-04-07 02:06:17  295324092 roberta-base-100.tar.gz
2021-04-07 03:22:42  295324092 roberta-base-1000.tar.gz
2021-04-07 03:22:47  295324092 roberta-base-1001.tar.gz
2021-04-07 03:22:51  295324092 roberta-base-1002.tar.gz
2021-04-07 03:22:54  295324092 roberta-base-1003.tar.gz
2021-04-07 03:22:56  295324092 roberta-base-1004.tar.gz
2021-04-07 03:23:06  295324092 roberta-base-1005.tar.gz
2021-04-07 03:23:15  295324092 roberta-base-1006.tar.gz
2021-04-07 03:23:17  295324092 roberta-base-1007.tar.gz
2021-04-07 03:23:23  295324092 roberta-base-1008.tar.gz
2021-04-07 03:23:25  295324092 roberta-base-1009.tar.gz
2021-04-07 02:06:23  295324092 roberta-base-101.tar.gz
2021-04-07 03:23:27  295324092 roberta-base-1010.tar.gz
2021-04-07 03:23:36  295324092 roberta-base-1011.tar.gz
2021-04-07 03:23:38  295324092 roberta-base-1012.tar.gz
20

2021-04-07 03:37:13  295324092 roberta-base-1176.tar.gz
2021-04-07 03:37:19  295324092 roberta-base-1177.tar.gz
2021-04-07 03:37:21  295324092 roberta-base-1178.tar.gz
2021-04-07 03:37:24  295324092 roberta-base-1179.tar.gz
2021-04-07 02:07:35  295324092 roberta-base-118.tar.gz
2021-04-07 03:37:31  295324092 roberta-base-1180.tar.gz
2021-04-07 03:37:35  295324092 roberta-base-1181.tar.gz
2021-04-07 03:37:38  295324092 roberta-base-1182.tar.gz
2021-04-07 03:37:41  295324092 roberta-base-1183.tar.gz
2021-04-07 03:37:44  295324092 roberta-base-1184.tar.gz
2021-04-07 03:37:50  295324092 roberta-base-1185.tar.gz
2021-04-07 03:37:59  295324092 roberta-base-1186.tar.gz
2021-04-07 03:38:02  295324092 roberta-base-1187.tar.gz
2021-04-07 03:38:05  295324092 roberta-base-1188.tar.gz
2021-04-07 03:38:09  295324092 roberta-base-1189.tar.gz
2021-04-07 02:07:40  295324092 roberta-base-119.tar.gz
2021-04-07 03:38:11  295324092 roberta-base-1190.tar.gz
2021-04-07 03:38:14  295324092 roberta-base-1191.t

2021-04-07 04:16:48  295324092 roberta-base-1636.tar.gz
2021-04-07 04:16:50  295324092 roberta-base-1637.tar.gz
2021-04-07 04:16:56  295324092 roberta-base-1638.tar.gz
2021-04-07 04:17:07  295324092 roberta-base-1639.tar.gz
2021-04-07 02:11:37  295324092 roberta-base-164.tar.gz
2021-04-07 04:17:10  295324092 roberta-base-1640.tar.gz
2021-04-07 04:17:13  295324092 roberta-base-1641.tar.gz
2021-04-07 04:17:16  295324092 roberta-base-1642.tar.gz
2021-04-07 04:17:20  295324092 roberta-base-1643.tar.gz
2021-04-07 04:17:22  295324092 roberta-base-1644.tar.gz
2021-04-07 04:17:26  295324092 roberta-base-1645.tar.gz
2021-04-07 04:17:29  295324092 roberta-base-1646.tar.gz
2021-04-07 04:17:35  295324092 roberta-base-1647.tar.gz
2021-04-07 04:17:38  295324092 roberta-base-1648.tar.gz
2021-04-07 04:17:41  295324092 roberta-base-1649.tar.gz
2021-04-07 02:11:43  295324092 roberta-base-165.tar.gz
2021-04-07 04:17:43  295324092 roberta-base-1650.tar.gz
2021-04-07 04:17:46  295324092 roberta-base-1651.t

2021-04-07 04:40:08  295324092 roberta-base-1899.tar.gz
2021-04-07 01:59:47  295324092 roberta-base-19.tar.gz
2021-04-07 02:13:48  295324092 roberta-base-190.tar.gz
2021-04-07 04:40:12  295324092 roberta-base-1900.tar.gz
2021-04-07 04:40:15  295324092 roberta-base-1901.tar.gz
2021-04-07 04:40:22  295324092 roberta-base-1902.tar.gz
2021-04-07 04:40:31  295324092 roberta-base-1903.tar.gz
2021-04-07 04:40:35  295324092 roberta-base-1904.tar.gz
2021-04-07 04:40:42  295324092 roberta-base-1905.tar.gz
2021-04-07 04:40:46  295324092 roberta-base-1906.tar.gz
2021-04-07 04:40:49  295324092 roberta-base-1907.tar.gz
2021-04-07 04:40:51  295324092 roberta-base-1908.tar.gz
2021-04-07 04:40:56  295324092 roberta-base-1909.tar.gz
2021-04-07 02:13:50  295324092 roberta-base-191.tar.gz
2021-04-07 04:41:02  295324092 roberta-base-1910.tar.gz
2021-04-07 04:41:09  295324092 roberta-base-1911.tar.gz
2021-04-07 04:41:14  295324092 roberta-base-1912.tar.gz
2021-04-07 04:41:17  295324092 roberta-base-1913.tar

2021-04-07 05:16:38  295324092 roberta-base-2303.tar.gz
2021-04-07 05:16:47  295324092 roberta-base-2304.tar.gz
2021-04-07 05:16:55  295324092 roberta-base-2305.tar.gz
2021-04-07 05:17:01  295324092 roberta-base-2306.tar.gz
2021-04-07 05:17:03  295324092 roberta-base-2307.tar.gz
2021-04-07 05:17:06  295324092 roberta-base-2308.tar.gz
2021-04-07 05:17:14  295324092 roberta-base-2309.tar.gz
2021-04-07 02:18:04  295324092 roberta-base-231.tar.gz
2021-04-07 05:17:18  295324092 roberta-base-2310.tar.gz
2021-04-07 05:17:26  295324092 roberta-base-2311.tar.gz
2021-04-07 05:17:28  295324092 roberta-base-2312.tar.gz
2021-04-07 05:17:36  295324092 roberta-base-2313.tar.gz
2021-04-07 05:17:47  295324092 roberta-base-2314.tar.gz
2021-04-07 05:17:58  295324092 roberta-base-2315.tar.gz
2021-04-07 05:18:01  295324092 roberta-base-2316.tar.gz
2021-04-07 05:18:09  295324092 roberta-base-2317.tar.gz
2021-04-07 05:18:19  295324092 roberta-base-2318.tar.gz
2021-04-07 05:18:21  295324092 roberta-base-2319.

2021-04-07 06:01:41  295324092 roberta-base-2765.tar.gz
2021-04-07 06:01:48  295324092 roberta-base-2766.tar.gz
2021-04-07 06:01:56  295324092 roberta-base-2767.tar.gz
2021-04-07 06:01:59  295324092 roberta-base-2768.tar.gz
2021-04-07 06:02:07  295324092 roberta-base-2769.tar.gz
2021-04-07 02:21:59  295324092 roberta-base-277.tar.gz
2021-04-07 06:02:14  295324092 roberta-base-2770.tar.gz
2021-04-07 06:02:24  295324092 roberta-base-2771.tar.gz
2021-04-07 06:02:34  295324092 roberta-base-2772.tar.gz
2021-04-07 06:02:41  295324092 roberta-base-2773.tar.gz
2021-04-07 06:02:50  295324092 roberta-base-2774.tar.gz
2021-04-07 06:02:52  295324092 roberta-base-2775.tar.gz
2021-04-07 06:02:54  295324092 roberta-base-2776.tar.gz
2021-04-07 06:02:57  295324092 roberta-base-2777.tar.gz
2021-04-07 06:03:04  295324092 roberta-base-2778.tar.gz
2021-04-07 06:03:07  295324092 roberta-base-2779.tar.gz
2021-04-07 02:22:03  295324092 roberta-base-278.tar.gz
2021-04-07 06:03:14  295324092 roberta-base-2780.t

2021-04-07 06:19:25  295324092 roberta-base-2950.tar.gz
2021-04-07 06:19:28  295324092 roberta-base-2951.tar.gz
2021-04-07 06:19:30  295324092 roberta-base-2952.tar.gz
2021-04-07 06:19:33  295324092 roberta-base-2953.tar.gz
2021-04-07 06:19:42  295324092 roberta-base-2954.tar.gz
2021-04-07 06:19:47  295324092 roberta-base-2955.tar.gz
2021-04-07 06:19:58  295324092 roberta-base-2956.tar.gz
2021-04-07 06:20:03  295324092 roberta-base-2957.tar.gz
2021-04-07 06:20:06  295324092 roberta-base-2958.tar.gz
2021-04-07 06:20:13  295324092 roberta-base-2959.tar.gz
2021-04-07 02:23:32  295324092 roberta-base-296.tar.gz
2021-04-07 06:20:24  295324092 roberta-base-2960.tar.gz
2021-04-07 06:20:27  295324092 roberta-base-2961.tar.gz
2021-04-07 06:20:35  295324092 roberta-base-2962.tar.gz
2021-04-07 06:20:44  295324092 roberta-base-2963.tar.gz
2021-04-07 06:20:56  295324092 roberta-base-2964.tar.gz
2021-04-07 06:20:59  295324092 roberta-base-2965.tar.gz
2021-04-07 06:21:10  295324092 roberta-base-2966.

2021-04-07 02:57:48  295324092 roberta-base-707.tar.gz
2021-04-07 02:57:53  295324092 roberta-base-708.tar.gz
2021-04-07 02:57:59  295324092 roberta-base-709.tar.gz
2021-04-07 02:03:46  295324092 roberta-base-71.tar.gz
2021-04-07 02:58:02  295324092 roberta-base-710.tar.gz
2021-04-07 02:58:04  295324092 roberta-base-711.tar.gz
2021-04-07 02:58:07  295324092 roberta-base-712.tar.gz
2021-04-07 02:58:11  295324092 roberta-base-713.tar.gz
2021-04-07 02:58:21  295324092 roberta-base-714.tar.gz
2021-04-07 02:58:25  295324092 roberta-base-715.tar.gz
2021-04-07 02:58:34  295324092 roberta-base-716.tar.gz
2021-04-07 02:58:38  295324092 roberta-base-717.tar.gz
2021-04-07 02:58:44  295324092 roberta-base-718.tar.gz
2021-04-07 02:58:49  295324092 roberta-base-719.tar.gz
2021-04-07 02:03:49  295324092 roberta-base-72.tar.gz
2021-04-07 02:58:51  295324092 roberta-base-720.tar.gz
2021-04-07 02:58:53  295324092 roberta-base-721.tar.gz
2021-04-07 02:59:01  295324092 roberta-base-722.tar.gz
2021-04-07 0

# SageMaker endpoint deployment

In [13]:
%%sh

# The name of our algorithm
algorithm_name=nlp-sagemaker-multimodel

cd container

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
region=${region:-us-west-2}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
$(aws ecr get-login --region ${region} --no-include-email)

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

docker build -q -t ${algorithm_name} .
docker tag ${algorithm_name} ${fullname}

docker push ${fullname}

Login Succeeded
sha256:977f37177f5011d8e7aab21919bb4da0671afc71c4ca4e48b9947efa849edcf4
The push refers to repository [815580386319.dkr.ecr.us-west-2.amazonaws.com/nlp-sagemaker-multimodel]
7c8cf815a4ad: Preparing
f50fba7a6e7d: Preparing
2000a4e77c14: Preparing
e13123642757: Preparing
60c492d92370: Preparing
fd67fd83d5a3: Preparing
5b2adbb59683: Preparing
53f49e7e2063: Preparing
a25f01bce16d: Preparing
6f15325cc380: Preparing
1e77dd81f9fa: Preparing
030309cad0ba: Preparing
5b2adbb59683: Waiting
53f49e7e2063: Waiting
a25f01bce16d: Waiting
fd67fd83d5a3: Waiting
6f15325cc380: Waiting
e13123642757: Layer already exists
f50fba7a6e7d: Layer already exists
7c8cf815a4ad: Layer already exists
2000a4e77c14: Layer already exists
60c492d92370: Layer already exists
fd67fd83d5a3: Layer already exists
5b2adbb59683: Layer already exists
6f15325cc380: Layer already exists
a25f01bce16d: Layer already exists
53f49e7e2063: Layer already exists
1e77dd81f9fa: Layer already exists
030309cad0ba: Layer already

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



In [14]:
import boto3
from sagemaker import get_execution_role

sm_client = boto3.client(service_name='sagemaker')
runtime_sm_client = boto3.client(service_name='sagemaker-runtime')

account_id = boto3.client('sts').get_caller_identity()['Account']
region = boto3.Session().region_name

s3_bucket = 'eks-sagemaker-us-west-2-bucket'

role = get_execution_role()

In [15]:
from time import gmtime, strftime

model_name = 'roberta-multimodel-' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
model_url = 's3://{}/roberta/'.format(s3_bucket) ## MODEL S3 URL
container = '{}.dkr.ecr.{}.amazonaws.com/nlp-sagemaker-multimodel:latest'.format(account_id, region)
instance_type = 'ml.c5d.18xlarge'

print('Model name: ' + model_name)
print('Model data Url: ' + model_url)
print('Container image: ' + container)

container = {
    'Image': container,
    'ModelDataUrl': model_url,
    'Mode': 'MultiModel'
}

create_model_response = sm_client.create_model(
    ModelName = model_name,
    ExecutionRoleArn = role,
    Containers = [container])

print("Model Arn: " + create_model_response['ModelArn'])

Model name: roberta-multimodel-2021-04-07-15-59-49
Model data Url: s3://eks-sagemaker-us-west-2-bucket/roberta/
Container image: 815580386319.dkr.ecr.us-west-2.amazonaws.com/nlp-sagemaker-multimodel:latest
Model Arn: arn:aws:sagemaker:us-west-2:815580386319:model/roberta-multimodel-2021-04-07-15-59-49


In [16]:
endpoint_config_name = 'roberta-multimodel-config' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print('Endpoint config name: ' + endpoint_config_name)

create_endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName = endpoint_config_name,
    ProductionVariants=[{
        'InstanceType': instance_type,
        'InitialInstanceCount': 1,
        'InitialVariantWeight': 1,
        'ModelName': model_name,
        'VariantName': 'AllTraffic'}])

print("Endpoint config Arn: " + create_endpoint_config_response['EndpointConfigArn'])

Endpoint config name: roberta-multimodel-config2021-04-07-15-59-50
Endpoint config Arn: arn:aws:sagemaker:us-west-2:815580386319:endpoint-config/roberta-multimodel-config2021-04-07-15-59-50


In [17]:
%%time

import time

endpoint_name = 'roberta-multimodel-endpoint' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print('Endpoint name: ' + endpoint_name)

create_endpoint_response = sm_client.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_config_name)
print('Endpoint Arn: ' + create_endpoint_response['EndpointArn'])

resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
status = resp['EndpointStatus']
print("Endpoint Status: " + status)

print('Waiting for {} endpoint to be in service...'.format(endpoint_name))
waiter = sm_client.get_waiter('endpoint_in_service')
waiter.wait(EndpointName=endpoint_name)

Endpoint name: roberta-multimodel-endpoint2021-04-07-15-59-50
Endpoint Arn: arn:aws:sagemaker:us-west-2:815580386319:endpoint/roberta-multimodel-endpoint2021-04-07-15-59-50
Endpoint Status: Creating
Waiting for roberta-multimodel-endpoint2021-04-07-15-59-50 endpoint to be in service...
CPU times: user 206 ms, sys: 11.7 ms, total: 218 ms
Wall time: 7min 31s


In [ ]:
import numpy as np 
import datetime
import math
import time
import matplotlib.pyplot as plt
import random

from essential_generators import DocumentGenerator
from sagemaker.serializers import JSONSerializer
gen = DocumentGenerator()
gen.init_word_cache(5000)
gen.init_sentence_cache(5000)

total_runs = 500
max_models = 30

content_type = "application/json" 
accept = "application/octet-stream"

print('Running {} inferences for {} (max models: {}):'.format(total_runs, endpoint_name, max_models))

client_times = []
errors_list = []

cw_start = datetime.datetime.utcnow()

errors = 0

print(gen.sentence())

for _ in range(total_runs):
    target_model = f'roberta-base-{random.randint(0,max_models)}.tar.gz'
    test_data = {"text": gen.sentence()}
    jsons = JSONSerializer()
    payload = jsons.serialize(test_data)
    client_start = time.time()
    response = runtime_sm_client.invoke_endpoint(
        EndpointName=endpoint_name,
        ContentType=content_type,
        TargetModel=target_model,
        Body=payload)
    client_end = time.time()
    client_times.append((client_end - client_start)*1000)
    skunk = response['Body'].read()
    #print(len(skunk))  

cw_end = datetime.datetime.utcnow()    

cw_duration = cw_end - cw_start 
duration_in_s = cw_duration.total_seconds() 

tps = total_runs/duration_in_s

print('\nErrors - {:.4f} out of {:.4f} total runs | {:.4f}% in {:.4f} seconds \n'.format(errors, total_runs, (errors/total_runs)*100, duration_in_s))
errors = 0

print('\nTPS: {:.4f}'.format(tps))
    
print('Client end-to-end latency percentiles:')
client_avg = np.mean(client_times)
client_p50 = np.percentile(client_times, 50)
client_p90 = np.percentile(client_times, 90)
client_p95 = np.percentile(client_times, 95)
client_p100 = np.percentile(client_times, 100)
print('Avg | P50 | P90 | P95 | P100')
print('{:.4f} | {:.4f} | {:.4f} | {:.4f} | {:.4f} \n'.format(client_avg, client_p50, client_p90, client_p95, client_p100))

# Give 5 minute buffer to end
cw_end += datetime.timedelta(minutes=5)

Running 500 inferences for roberta-multimodel-endpoint2021-04-07-15-59-50 (max models: 30):
Hotel" or primarily Protestant
